<a href="https://colab.research.google.com/github/Talha1818/GPT-Text-Generation-LLM/blob/master/Gpt2_Text_Generation_AutoModelForCausalLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import string
import re

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Final_Text_Dataset_22_May_2023.csv")
df['token'] = df['token'].astype(int)
df.head()

,page#,package_name,text,token,clean_text
0,1.0,611438_NJ_BURLINGTON,"HADDONFIELD - CITY ABSTRACT, L 1LBS 10F1\nZaSS...",15,haddonfield city abstract lbs zassokaeib name ...
1,2.0,611438_NJ_BURLINGTON,"oO ANOrIRE: REDSEiY. SRARIT U Bsc, Hh Mea, ie ...",15,anorire redseiy srarit bsc mea laa wit bth ipe...
2,3.0,611438_NJ_BURLINGTON,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,city abstract llc alta buyers settlement state...
3,4.0,611438_NJ_BURLINGTON,po Description Bayer\nTitle - Lender's Title P...,11,description bayer title lenders title policy c...
4,5.0,611438_NJ_BURLINGTON,"City Abstract, LLC ALTA Seller's Settlement St...",13,city abstract llc alta sellers settlement stat...


In [ ]:
# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# df['clean_text'] = "<SOD> " + df['clean_text'] + " [EOD]"

df = df[df['clean_text'].notna()]
df = df[df['token'].notna()]

In [ ]:
df.head()

,page#,package_name,text,token,clean_text
0,1.0,611438_NJ_BURLINGTON,"HADDONFIELD - CITY ABSTRACT, L 1LBS 10F1\nZaSS...",15,haddonfield city abstract lbs zassokaeib name ...
1,2.0,611438_NJ_BURLINGTON,"oO ANOrIRE: REDSEiY. SRARIT U Bsc, Hh Mea, ie ...",15,anorire redseiy srarit bsc mea laa wit bth ipe...
2,3.0,611438_NJ_BURLINGTON,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,city abstract llc alta buyers settlement state...
3,4.0,611438_NJ_BURLINGTON,po Description Bayer\nTitle - Lender's Title P...,11,description bayer title lenders title policy c...
4,5.0,611438_NJ_BURLINGTON,"City Abstract, LLC ALTA Seller's Settlement St...",13,city abstract llc alta sellers settlement stat...


In [ ]:
# df['text'] = '<startofstring> ' + df['clean_text'].astype(str) + ' <bot>: ' + df['token'].astype(str) + ' <endofstring>'

In [ ]:
# df['text'][0]

In [ ]:
df1 = df['token'].value_counts().to_frame().reset_index().rename(columns={'index':'token','token':'count'})
df_100 = df1[df1['count']>=100]['token'].tolist()
df_new = df[df['token'].isin(df_100)]
len(df_new['token'].unique())

130

In [ ]:
df = df_new[['clean_text', 'token']].copy()
df.head()

,clean_text,token
0,haddonfield city abstract lbs zassokaeib name ...,15
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15
2,city abstract llc alta buyers settlement state...,11
3,description bayer title lenders title policy c...,11
4,city abstract llc alta sellers settlement stat...,13


In [ ]:
df['token'].value_counts()[:5].index.tolist()

[15, 14, 30, 28, 40]

In [ ]:
# df = df[df['token'].isin(df['token'].value_counts()[:5].index.tolist())]
# df.shape

In [ ]:
df_token = pd.read_excel("/content/drive/MyDrive/Annotation Folder/field_tokens.xlsx", sheet_name='Lender_field_tokens')
df_token = df_token.rename(columns={'Token.1': 'Meaning', 'Token':'token'})
df_token = df_token[['token', 'Meaning']]
df_token.head(2)

,token,Meaning
0,0,Cover Sheet
1,1,MARKED UP COMMITMENT


In [ ]:
df.head(2)

,clean_text,token
0,haddonfield city abstract lbs zassokaeib name ...,15
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15


In [ ]:
# Merge the dataframes based on the 'Token' column
merged_df = pd.merge(df, df_token, on='token')
merged_df.head()

,clean_text,token,Meaning
0,haddonfield city abstract lbs zassokaeib name ...,15,LENDER’S INSTRUCTIONS
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15,LENDER’S INSTRUCTIONS
2,instructions escrowtitleclosing agent uuuuu lo...,15,LENDER’S INSTRUCTIONS
3,loan alta policy endorsements issued within da...,15,LENDER’S INSTRUCTIONS
4,loan signatures borrowers must sign documents ...,15,LENDER’S INSTRUCTIONS


In [ ]:
def remove_punctuation_and_lower(text):
    # Define punctuation excluding comma
    punctuation = string.punctuation.replace(',', '')
    
    # Remove punctuation excluding comma
    translator = str.maketrans('', '', punctuation)
    text_without_punct = text.translate(translator)
    
    # Remove apostrophe
    text_without_punct = re.sub(r"[’']", '', text_without_punct)
    
    # Convert to lowercase
    text_lower = text_without_punct.lower()
    
    return text_lower

In [ ]:
remove_punctuation_and_lower("LENDER’S INSTRUCTIONS")

'lenders instructions'

In [ ]:
df = merged_df.copy()

In [ ]:
df['Meaning'] = df['Meaning'].apply(remove_punctuation_and_lower)
df.head()

,clean_text,token,Meaning
0,haddonfield city abstract lbs zassokaeib name ...,15,lenders instructions
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15,lenders instructions
2,instructions escrowtitleclosing agent uuuuu lo...,15,lenders instructions
3,loan alta policy endorsements issued within da...,15,lenders instructions
4,loan signatures borrowers must sign documents ...,15,lenders instructions


In [ ]:
df['Meaning'] = 'label: ' + df['Meaning']
df.head()

,clean_text,token,Meaning
0,haddonfield city abstract lbs zassokaeib name ...,15,label: lenders instructions
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15,label: lenders instructions
2,instructions escrowtitleclosing agent uuuuu lo...,15,label: lenders instructions
3,loan alta policy endorsements issued within da...,15,label: lenders instructions
4,loan signatures borrowers must sign documents ...,15,label: lenders instructions


In [ ]:
df['Meaning'].value_counts().to_frame()

,Meaning
label: lenders instructions,6874
label: buyers cd,5954
label: lender mortgage,5172
label: lender note,4798
label: lender amortizationpayment schedule,4716
...,...
label: contact information authorization,103
label: lender mpf covid19 certification,103
label: lender home inspection,102
label: lender closing disclosure addendum,101


In [ ]:
# df_token[df_token['Token'].isin(df['token'].value_counts()[:5].index.tolist())]['Meaning']

In [ ]:
# df['token'] = df['token'].map({14:"BUYERS CD", 15:"LENDERS INSTRUCTIONS",
#                                28:"Lender Note", 40:"Lender Amortization/Payment Schedule"})
# df.head()

In [ ]:
df['text'] = '<startofstring> ' + df['clean_text'].astype(str) + ' <bot>: ' + df['Meaning'].astype(str) + ' <endofstring>'

In [ ]:
df.head()

,clean_text,token,Meaning,text
0,haddonfield city abstract lbs zassokaeib name ...,15,label: lenders instructions,<startofstring> haddonfield city abstract lbs ...
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15,label: lenders instructions,<startofstring> anorire redseiy srarit bsc mea...
2,instructions escrowtitleclosing agent uuuuu lo...,15,label: lenders instructions,<startofstring> instructions escrowtitleclosin...
3,loan alta policy endorsements issued within da...,15,label: lenders instructions,<startofstring> loan alta policy endorsements ...
4,loan signatures borrowers must sign documents ...,15,label: lenders instructions,<startofstring> loan signatures borrowers must...


In [ ]:
df['text'][0]

'<startofstring> haddonfield city abstract lbs zassokaeib name dwr hie republic first bank dba republic route north suite marlton yeie ron billing <bot>: label: lenders instructions <endofstring>'

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW

In [ ]:
from datasets import load_dataset

# Step 1: Prepare your dataset
dataset = load_dataset('imdb')  # Example dataset, replace with your own

dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
from sklearn.model_selection import train_test_split


train_df, valid_df = train_test_split(
    df,
    test_size=0.25,
    random_state=2023
)

In [ ]:
from datasets import Dataset, DatasetDict

# Converting the DataFrames into Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Creating the final DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset,
})

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'token', 'Meaning', 'text', '__index_level_0__'],
        num_rows: 66413
    })
    valid: Dataset({
        features: ['clean_text', 'token', 'Meaning', 'text', '__index_level_0__'],
        num_rows: 22138
    })
})

In [ ]:
# Step 2: Load and tokenize the data
tokenizer = AutoTokenizer.from_pretrained('gpt2')  # Or any other GPT model
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

tokenized_data = tokenizer.batch_encode_plus(dataset_dict['train']['text'], padding="max_length", truncation=True, max_length=256)

# Step 3: Encode the data
input_ids = torch.tensor(tokenized_data['input_ids'])
attention_mask = torch.tensor(tokenized_data['attention_mask'])
# labels = torch.tensor(dataset_dict['train']['token'])

In [ ]:
df.shape

(88551, 4)

In [ ]:
88551/16

5534.4375

In [ ]:
# Step 4: Prepare data loaders
batch_size = 8
# train_dataset = torch.utils.data.TensorDataset(input_ids[:10000], attention_mask[:10000], labels[:10000])
train_dataset = torch.utils.data.TensorDataset(input_ids, attention_mask)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_tokenized_data = tokenizer.batch_encode_plus(dataset_dict['valid']['text'], padding='max_length', truncation=True, max_length=256)
test_input_ids = torch.tensor(test_tokenized_data['input_ids'])
test_attention_mask = torch.tensor(test_tokenized_data['attention_mask'])
# test_labels = torch.tensor(dataset_dict['valid']['token'][:10000])

# test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_labels)
test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Step 6: Load the GPT model for sequence classification
model = AutoModelForCausalLM.from_pretrained('gpt2')  # Or any other GPT model

# Attach a classification head to the model
# classification_head = torch.nn.Linear(model.config.hidden_size, 2)  # 2 classes (e.g., positive/negative)
model.resize_token_embeddings(len(tokenizer))
# model.add_module("classification_head", classification_head)

Embedding(50261, 768)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        input_ids_batch, attention_mask_batch = batch

        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=input_ids_batch)  # Use input_ids_batch as labels
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {average_loss:.4f}")

    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids_batch, attention_mask_batch = batch

            input_ids_batch = input_ids_batch.to(device)
            attention_mask_batch = attention_mask_batch.to(device)

            outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=input_ids_batch)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()
    
    average_val_loss = total_val_loss / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {average_val_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2 - Loss: 3.4400
Epoch 1/2 - Validation Loss: 2.1336
Epoch 2/2 - Loss: 1.9790
Epoch 2/2 - Validation Loss: 1.5825


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# num_epochs = 1
# for epoch in range(num_epochs):
#     model.eval()  # Set the model to evaluation mode
#     total_val_loss = 0
    
#     with torch.no_grad():
#         for batch in test_loader:
#             input_ids_batch, attention_mask_batch = batch

#             input_ids_batch = input_ids_batch.to(device)
#             attention_mask_batch = attention_mask_batch.to(device)

#             outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=input_ids_batch)
#             val_loss = outputs.loss
#             total_val_loss += val_loss.item()
    
#     average_val_loss = total_val_loss / len(test_loader)
#     print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {average_val_loss:.4f}")


Epoch 1/1 - Validation Loss: 2.1545


In [ ]:
device

device(type='cuda')

In [ ]:
# Step 8: Save the trained model
model.save_pretrained("/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs")

('/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/tokenizer_config.json',
 '/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/special_tokens_map.json',
 '/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/vocab.json',
 '/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/merges.txt',
 '/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/added_tokens.json',
 '/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs')
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [ ]:
prompt = "statefinanc matthew zimnoch janine lumoya oakwood unit penn valley loan may concern matthew zimnoch verify social security number janine lumoya verify social security number asly obtained credit social security number wan date state financial network llc hillman drive suite chadds ford nmls"

In [ ]:
prompt_inp = "<startofstring> " + prompt + " <bot>:"
max_length = 256  # Maximum number of tokens to generate
temperature = 0.2  # Controls the randomness of token generation

# Set pad_token_id to eos_token_id
model.config.pad_token_id = model.config.eos_token_id

# Encode the prompt and generate the next sequence of tokens
input_ids = tokenizer.encode(prompt_inp, return_tensors='pt').to(device)

input_ids = input_ids[:, :max_length]  # Truncate or pad the input to max_length if needed
output = model.generate(input_ids=input_ids, max_length=max_length, temperature=temperature)

# Decode the generated tokens back into text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Find the index of "<bot>:" and extract the text after it
index = generated_text.find("<bot>:")
extracted_text = generated_text[index + len("<bot>:"):].strip()

print(extracted_text)

label: lender social security number


In [ ]:
valid_df.head()

,clean_text,token,Meaning,text
3104,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...
30146,loan min fha case note may melville new york d...,28,label: lender note,<startofstring> loan min fha case note may mel...
35089,lender bank borrowers sean lacy taylor lacy pr...,22,label: lender compliance agreement,<startofstring> lender bank borrowers sean lac...
77358,loan know home leadbased paint appraisal deter...,223,label: notice to purchaser,<startofstring> loan know home leadbased paint...
6127,mortgage solutions colorado llc document date ...,15,label: lenders instructions,<startofstring> mortgage solutions colorado ll...


In [ ]:
df['text'][102160]

'<startofstring> statefinanc matthew zimnoch janine lumoya oakwood unit penn valley loan may concern matthew zimnoch verify social security number janine lumoya verify social security number asly obtained credit social security number wan date state financial network llc hillman drive suite chadds ford nmls <bot>: Lender Note <endofstring>'

# **Model Inference**

In [ ]:
def prediction_update(text, max_length=300, temperature=0.1, top_k=0, top_p=0.9):
    prompt = text
    prompt_inp = "<startofstring> " + prompt + " <bot>:"

    model.config.pad_token_id = model.config.eos_token_id

    encoding = tokenizer.encode_plus(prompt_inp, padding='max_length', truncation=True, max_length=256, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )

    # Decode the generated tokens back into text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Find the index of "<bot>:" and extract the text after it
    index = generated_text.find("<bot>:")
    extracted_text = generated_text[index + len("<bot>:"):].strip()

    return extracted_text


In [ ]:
test_df = valid_df.copy()

In [ ]:
test_df['text_len'] = test_df['clean_text'].apply(len)

In [ ]:
prediction_update(test_df['clean_text'][3104])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


'label: lenders instructions'

In [ ]:
test_df.head()

,clean_text,token,Meaning,text,text_len
3104,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...,151
30146,loan min fha case note may melville new york d...,28,label: lender note,<startofstring> loan min fha case note may mel...,2385
35089,lender bank borrowers sean lacy taylor lacy pr...,22,label: lender compliance agreement,<startofstring> lender bank borrowers sean lac...,1353
77358,loan know home leadbased paint appraisal deter...,223,label: notice to purchaser,<startofstring> loan know home leadbased paint...,1402
6127,mortgage solutions colorado llc document date ...,15,label: lenders instructions,<startofstring> mortgage solutions colorado ll...,1864


In [ ]:
test_df[test_df['text_len']>6000]

,clean_text,token,Meaning,text,text_len
59580,loan borrower fails maintain coverages describ...,30,label: lender mortgage,<startofstring> loan borrower fails maintain c...,6440
39502,loan form rev page cases following person must...,32,label: lenders borrowers w9,<startofstring> loan form rev page cases follo...,6865
60138,loan sums secured security instrument borrower...,30,label: lender mortgage,<startofstring> loan sums secured security ins...,6295
40550,loan form rev page cases following person must...,32,label: lenders borrowers w9,<startofstring> loan form rev page cases follo...,6847
42434,loan form rev page cases following person must...,32,label: lenders borrowers w9,<startofstring> loan form rev page cases follo...,6798
...,...,...,...,...,...
42560,loan form rev page line exemptions ugtae commo...,32,label: lenders borrowers w9,<startofstring> loan form rev page line exempt...,6138
56770,loan determination certification tracking serv...,30,label: lender mortgage,<startofstring> loan determination certificati...,6467
38958,loan form rev page cases following person must...,32,label: lenders borrowers w9,<startofstring> loan form rev page cases follo...,6838
57763,loan sums secured security instrument borrower...,30,label: lender mortgage,<startofstring> loan sums secured security ins...,6279


In [ ]:
test_df_ = test_df[test_df['text_len']<1000]
test_df_.head()

,clean_text,token,Meaning,text,text_len
3104,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...,151
15389,addendum closing disclosure form continued sta...,14,label: buyers cd,<startofstring> addendum closing disclosure fo...,261
55892,american land title association commitment tit...,1,label: marked up commitment,<startofstring> american land title associatio...,909
45243,amortization schedule loan date borrowers nich...,40,label: lender amortizationpayment schedule,<startofstring> amortization schedule loan dat...,309
10158,acknowledgement wei carefully reviewed settlem...,13,label: alta sellers statement,<startofstring> acknowledgement wei carefully ...,389


In [ ]:
prediction_update(test_df['clean_text'][10158])

'label: alta sellers statement'

In [ ]:
# test_df['clean_text'][30146]

In [ ]:
# valid_df['prediction'] = valid_df['clean_text'].apply(prediction_update)

In [ ]:
# valid_df.head()

In [ ]:
from tqdm.auto import tqdm
y_pred = []
for i in tqdm(test_df_['clean_text'], desc='processing'):
  y_pred.append(prediction_update(i))

processing:   0%|          | 0/7874 [00:00<?, ?it/s]

In [ ]:
y_pred[:5]

['label: lenders instructions',
 'label: buyers cd',
 'label: deed copy',
 'label: lender amortizationpayment schedule',
 'label: alta sellers statement']

In [ ]:
test_df_['LLM_prediction'] = y_pred

<ipython-input-53-cc72b7b7e4ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_['LLM_prediction'] = y_pred


In [ ]:
test_df_ = test_df_.reset_index(drop=True)

In [ ]:
test_df_.head()

,clean_text,token,Meaning,text,text_len,LLM_prediction
0,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...,151,label: lenders instructions
1,addendum closing disclosure form continued sta...,14,label: buyers cd,<startofstring> addendum closing disclosure fo...,261,label: buyers cd
2,american land title association commitment tit...,1,label: marked up commitment,<startofstring> american land title associatio...,909,label: deed copy
3,amortization schedule loan date borrowers nich...,40,label: lender amortizationpayment schedule,<startofstring> amortization schedule loan dat...,309,label: lender amortizationpayment schedule
4,acknowledgement wei carefully reviewed settlem...,13,label: alta sellers statement,<startofstring> acknowledgement wei carefully ...,389,label: alta sellers statement


In [ ]:
test_df_['Meaning'].nunique()

123

In [ ]:
cleaned_labels = []

i = 0
for label in tqdm(test_df_['LLM_prediction']):
  try:
    clean = label.split("label:")[1].strip()
    # print(label, "-->", clean)
    # print("ERRRO:", i)
    cleaned_labels.append(clean)
    i+=1
  except:
    cleaned_labels.append('')


  0%|          | 0/7874 [00:00<?, ?it/s]

In [ ]:
# test_df_['LLM_prediction'][66]

In [ ]:
test_df_['clean_prediction'] = cleaned_labels

In [ ]:
test_df__ = test_df_[~(test_df_['clean_prediction']=='')]
test_df__.shape

(7846, 7)

In [ ]:
test_df__.head()

,clean_text,token,Meaning,text,text_len,LLM_prediction,clean_prediction
0,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...,151,label: lenders instructions,lenders instructions
1,addendum closing disclosure form continued sta...,14,label: buyers cd,<startofstring> addendum closing disclosure fo...,261,label: buyers cd,buyers cd
2,american land title association commitment tit...,1,label: marked up commitment,<startofstring> american land title associatio...,909,label: deed copy,deed copy
3,amortization schedule loan date borrowers nich...,40,label: lender amortizationpayment schedule,<startofstring> amortization schedule loan dat...,309,label: lender amortizationpayment schedule,lender amortizationpayment schedule
4,acknowledgement wei carefully reviewed settlem...,13,label: alta sellers statement,<startofstring> acknowledgement wei carefully ...,389,label: alta sellers statement,alta sellers statement


In [ ]:
test_df__['actual_label'] = test_df__['Meaning'].apply(lambda x: x.split("label:")[1].strip())

<ipython-input-62-57f87d35a5ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df__['actual_label'] = test_df__['Meaning'].apply(lambda x: x.split("label:")[1].strip())


In [ ]:
test_df__.head()

,clean_text,token,Meaning,text,text_len,LLM_prediction,clean_prediction,actual_label
0,new jersey city abstract lbs ang hichway noat ...,15,label: lenders instructions,<startofstring> new jersey city abstract lbs a...,151,label: lenders instructions,lenders instructions,lenders instructions
1,addendum closing disclosure form continued sta...,14,label: buyers cd,<startofstring> addendum closing disclosure fo...,261,label: buyers cd,buyers cd,buyers cd
2,american land title association commitment tit...,1,label: marked up commitment,<startofstring> american land title associatio...,909,label: deed copy,deed copy,marked up commitment
3,amortization schedule loan date borrowers nich...,40,label: lender amortizationpayment schedule,<startofstring> amortization schedule loan dat...,309,label: lender amortizationpayment schedule,lender amortizationpayment schedule,lender amortizationpayment schedule
4,acknowledgement wei carefully reviewed settlem...,13,label: alta sellers statement,<startofstring> acknowledgement wei carefully ...,389,label: alta sellers statement,alta sellers statement,alta sellers statement


In [ ]:
test_df__.columns

Index(['clean_text', 'token', 'Meaning', 'text', 'text_len', 'LLM_prediction',
       'clean_prediction', 'actual_label'],
      dtype='object')

In [ ]:
test_df__.to_csv("/content/drive/MyDrive/Annotation Folder/LLM/LLM_results.csv", index=False)